In [ ]:
#setup the environment
!pip install -q openvino-dev[onnx,tensorflow2]==2022.1.0 &> error_log_openvino.txt
#!pip install -q tensorflow==2.5.1
!pip install -q pandas==1.2.4 &> error_log_panda.txt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

In [ ]:
tf.nn.softmax(predictions).numpy()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

In [ ]:
model_fname="mnist"
probability_model.save(model_fname)

In [ ]:
from openvino.inference_engine import IECore
from pathlib import Path
import json
import sys
import numpy as np
import matplotlib.pyplot as plt

# The paths of the source and converted models
model_name = "mnist"
model_path = Path(model_name)
ir_data_type = "FP16"
ir_model_name = "mnist_ir"

# Get the path to the Model Optimizer script

# Construct the command for Model Optimizer
mo_command = f"""mo
                 --saved_model_dir "{model_name}"
                 --input_shape "[1,28,28,1]" 
                 --data_type "{ir_data_type}" 
                 --output_dir "{model_path.parent}"
                 --model_name "{ir_model_name}"
                 """
mo_command = " ".join(mo_command.split())

# Run the Model Optimizer (overwrites the older model)
print("Exporting TensorFlow model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

In [ ]:
model_xml = "mnist_ir.xml"
model_bin = "mnist_ir.bin"

# Load network to the plugin
ie = IECore()
net = ie.read_network(model=model_xml, weights=model_bin)

exec_net = ie.load_network(network=net, device_name="CPU")

del net

#test against a few images from the dataset
input_list = x_test[:10]

for input_image in input_list:
  input_layer = next(iter(exec_net.input_info))
  output_layer = next(iter(exec_net.outputs))

  res = exec_net.infer(inputs={input_layer: input_image})
  res = res[output_layer]
  X = input_image
  X = X.reshape([28, 28]);
  plt.figure()
  plt.gray()
  plt.imshow(X)
  plt.text(0,-1, "The prediction is "+str(np.argmax(res[0]))+" @ "+str(max(res[0])*100)+"%")

## Let's Test the Tensorflow Solution on the test.csv!

In [ ]:
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
submit_data = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
test_imgs = test_data.to_numpy().reshape(28000,1, 28,28)

In [ ]:
test_imgs.shape

In [ ]:
pred = np.zeros((28000,1))
i=0
for test_img in test_imgs:
    input_layer = next(iter(exec_net.input_info))
    output_layer = next(iter(exec_net.outputs))
    res = exec_net.infer(inputs={input_layer: test_img})
    res = res[output_layer]
    #X = input_image
    #X = X.reshape([28, 28]);
    #plt.figure()
    #plt.gray()
    #plt.imshow(X)
    #plt.text(0,-1, "The prediction is "+str(np.argmax(res[0]))+" @ "+str(max(res[0])*100)+"%")
    pred[i] = np.argmax(res[0])
    i=i+1


In [ ]:
ind = pd.DataFrame(np.arange(1,28001), columns=['ImageId'])

pred_sub = pd.DataFrame(pred.astype('int32'), columns=['Label'])
submission = [ind , pred_sub]
sub = pd.concat(submission, axis=1)

compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
sub.to_csv('submission.zip', index=False, compression=compression_opts)

In [ ]:
pd.read_csv('submission.zip')